## 80. ID番号への変換
1. ~~単語をIDに変換する辞書を作成・保存(重いため,2回目以降はセル2を実行)~~
2. 辞書の読み込み
3. 単語列をID列に変換する関数

In [1]:
# pklファイルから読み込み
import pickle

with open('../data/ch09/name_to_id.pkl', 'rb') as tf:
    name_to_id = pickle.load(tf)

In [2]:
# 与えられた単語列に対し, ID番号の列を返す関数
# ch06ではCountVectorizerを利用したため, 今回もCountVectorizerを活用する
from sklearn.feature_extraction.text import CountVectorizer

def convert_words_to_ids(words):
    '''
    input :words(単語列)
    output:ids(ID番号列)
    '''
    # analyzer: 単語列に前処理を加え, listに変換する関数
    analyzer = CountVectorizer().build_analyzer()
    word_list = analyzer(words)
    
    ids = []
    for word in word_list:
        if word in name_to_id:
            ids.append(name_to_id[word])
        else:
            ids.append(0)  # 未知語の場合, IDを0とする
    
    return ids

## GPU prepare
1. 使用可能GPUの確認
2. GPUの指定
3. PyTorchで利用できるGPU数の確認

In [3]:
# 使用可能GPUの確認
!nvidia-smi

Wed Jun 15 12:45:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:01:00.0 Off |                  Off |
| 30%   28C    P8    17W / 300W |   2181MiB / 48685MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    On   | 00000000:25:00.0 Off |                  Off |
| 30%   

In [4]:
# GPUの指定
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2' #0番を使用するとき

In [5]:
# 確認
import torch
print(torch.cuda.device_count()) #Pytorchで使用できるGPU数を取得

1


## prepare
1. 語彙数の取得
2. 学習データの用意(ラベル)
3. 学習データの用意(特徴量)
4. 乱数の種を固定

In [6]:
# 語彙数の取得(ID:0の単語はまとめて1語とする), 未知語, paddingを考慮
vocab_size = max(name_to_id.values())+2

In [7]:
# 訓練・検証・評価データの用意
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

# ラベル: ch08の出力を利用
Y_train = np.loadtxt('../data/ch08/Y_train.txt')
Y_valid = np.loadtxt('../data/ch08/Y_valid.txt')
Y_test = np.loadtxt('../data/ch08/Y_test.txt')

# pytorch用に変換
Y_train_long = torch.tensor(Y_train, dtype=torch.int64)
Y_valid_long = torch.tensor(Y_valid, dtype=torch.int64)
Y_test_long = torch.tensor(Y_test, dtype=torch.int64)

In [8]:
# 特徴量: convert_words_to_ids(80)を利用
def convert_text_to_features(fname):
    '''
    input :fname
    output:features(tensor)
    '''
    with open(fname, encoding='utf-8') as f:
        lines = f.readlines()
    
    # id列(list)のリストに変換
    ids_list = [convert_words_to_ids(line) for line in lines]
    
    # id列(tensor)のリストに変換
    ids_tensor = [torch.tensor(ids, dtype=torch.int64) for ids in ids_list]
    
    # 最大のid+1(vocab_size-1)でパディング
    features = torch.nn.utils.rnn.pad_sequence(ids_tensor, batch_first=True, padding_value=vocab_size-1)
    
    return features

# 特徴量抽出
X_train_long = convert_text_to_features('../data/ch06/train.txt')
X_valid_long = convert_text_to_features('../data/ch06/valid.txt')
X_test_long = convert_text_to_features('../data/ch06/test.txt')

In [9]:
# 乱数シードの固定
import random

def fix_seed(seed):
    # random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

## 83. ミニバッチ化・GPU上での学習
1. ~~GPUに対応したRNNの定義~~
2. GPUに対応したaccuracy計測関数の定義
3. ~~ミニバッチ化し, GPU上で学習~~

In [10]:
# accuracyの計測
def measure_loss_accuracy2(model, criterion, dataloader):
    '''
    input : model, criterion, dataloader
    output: loss, accuracy
    '''
    running_loss = 0
    correct_num = 0
    device = model.device
    batch_size = model.batch_size
    for X, Y in dataloader:
        # GPU上に
        X = X.to(device)
        Y = Y.to(device)
        model.init_hidden()
        predict_y = model.forward(X)
        
        # lossの計算
        loss = criterion(predict_y, Y)
        running_loss += loss.item()
        
        # accuracyの計算
        predict_label = torch.max(predict_y, 1)[1]
        for i in range(batch_size):
            if predict_label[i] == Y[i]:
                correct_num += 1
    
    loss_avg = running_loss/len(dataloader)
    accuracy = correct_num/(len(dataloader)*batch_size)
    
    return loss_avg, accuracy

## 85. 双方向RNN・多層化
1. biRNNの定義
2. ~~biRNNの学習~~
3. ~~多層biRNNの学習~~

In [13]:
# biRNNの定義
class biRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size, batch_size, num_layers=1):
        # 層の定義
        super().__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.num_layers = num_layers
        
        self.emb = nn.Embedding(vocab_size, embedding_dim, padding_idx=vocab_size-1)
        # biRNNに対応: bidirectional=True
        self.birnn = nn.RNN(embedding_dim, hidden_size, num_layers=self.num_layers, batch_first=True, bidirectional=True)
        # biRNNに対応: birnnの出力次元はrnnの次元の2倍
        self.fc = nn.Linear(hidden_size*2, output_size)

        # GPUに移す
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self = self.to(self.device)
    
    def init_hidden(self, batch_size=None):
        if not batch_size:
            batch_size = self.batch_size
        # biRNNに対応: birnnの出力次元はrnnの次元の2倍
        self.hidden_state = torch.zeros(self.num_layers*2, batch_size, self.hidden_size).to(self.device)
    
    def forward(self, x):
        x = self.emb(x)
        # biRNNに対応
        x_birnn, self.hidden_state =self.birnn(x, self.hidden_state)
        x = self.fc(x_birnn[:,-1,:])
        return x

## 88. パラメータチューニング
1. train, validの定義
2. objectiveの定義
3. optunaによるパラメータチューニング
4. 結果表示

In [17]:
def train(model, device, train_loader, optimizer):
    model.train()
    for X, Y in train_loader:
        X, Y = X.to(device), Y.to(device)
        # forward
        optimizer.zero_grad()
        model.init_hidden()
        predict_y = model(X)
        loss = F.cross_entropy(predict_y, Y)
        # backward
        loss.backward()
        # 更新
        optimizer.step()

def valid(model, device, valid_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for X, Y in valid_loader:
            X, Y = X.to(device), Y.to(device)
            model.init_hidden()
            predict_y = model(X)
            predict_label = torch.max(predict_y, 1)[1]
            correct += predict_label.eq(Y.view_as(predict_label)).sum().item()
    return 1 - correct/len(valid_loader.dataset)

In [21]:
import torch.optim as optim

def objective(trial):
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    
    # RNNの層数
    num_layers = trial.suggest_int('num_layers', 1, 9)

    # hidden_size
    hidden_size = int(trial.suggest_discrete_uniform('hidden_size', 10, 150, 10))

    # batch_size
    batch_size = trial.suggest_int('batch_size', 8, 256)

    # lr
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)

    # dataloaderの定義
    train_dataset2 = torch.utils.data.TensorDataset(X_train_long, Y_train_long)
    train_dataloader2 = torch.utils.data.DataLoader(train_dataset2, batch_size=batch_size, drop_last=True)

    valid_dataset2 = torch.utils.data.TensorDataset(X_valid_long, Y_valid_long)
    valid_dataloader2 = torch.utils.data.DataLoader(valid_dataset2, batch_size=batch_size, drop_last=True)

    # モデルの作成
    model = biRNN(vocab_size, 300, hidden_size, 4, batch_size, num_layers)
    optimizer = optim.Adam(model.parameters(), lr)

    for epoch in range(20):
        train(model, device, train_dataloader2, optimizer)
        error_rate = valid(model, device, valid_dataloader2)
    
    return error_rate


In [23]:
import optuna

study = optuna.create_study()
study.optimize(objective, n_trials=50)

[I 2022-06-15 13:48:35,538] A new study created in memory with name: no-name-78929e7d-3f18-4301-b680-ead5da68a2a8
[I 2022-06-15 13:48:56,692] Trial 0 finished with value: 0.5958083832335329 and parameters: {'num_layers': 6, 'hidden_size': 40.0, 'batch_size': 185, 'lr': 0.02399012871230951}. Best is trial 0 with value: 0.5958083832335329.
[I 2022-06-15 13:49:50,211] Trial 1 finished with value: 0.6122754491017964 and parameters: {'num_layers': 4, 'hidden_size': 140.0, 'batch_size': 50, 'lr': 0.012192549200322086}. Best is trial 0 with value: 0.5958083832335329.
[I 2022-06-15 13:52:04,230] Trial 2 finished with value: 0.8817365269461078 and parameters: {'num_layers': 7, 'hidden_size': 50.0, 'batch_size': 36, 'lr': 0.09780523335514346}. Best is trial 0 with value: 0.5958083832335329.
[I 2022-06-15 13:52:57,258] Trial 3 finished with value: 0.6212574850299402 and parameters: {'num_layers': 7, 'hidden_size': 90.0, 'batch_size': 80, 'lr': 0.017394906930627063}. Best is trial 0 with value: 0.

In [24]:
print(f'best params: {study.best_params}')
print(f'best value: {study.best_value}')

best params: {'num_layers': 1, 'hidden_size': 90.0, 'batch_size': 26, 'lr': 0.0007633693373516085}
best value: 0.15568862275449102
